In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Change the path according to the location of your CSV file
file_path = '/content/drive/MyDrive/MeditaZone-Resources/Model & Data ML/Try Recommendation.csv'

# Read data from a csv file and create a DataFrame
df_first = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df_first.head()

,Output,Meditation,Quotes,Artikel,Audio
0,Anxiety,Mindfullness,"""Ketika pikiranmu terjerat di dalam labirin, i...","Anxiety Menyerang, Bagaimana Cara Meredakannya?",Consicious Living Meditation
1,Anxiety,Metta (kasih sayang),"""Ketika pikiranmu penuh dengan keraguan, ingat...",Generalized Anxiety Disorder (Gangguan Kecemas...,Body_s Harmony of Loving-Kindess
2,Anxiety,Meditasi Pernafasan (Breath Awareness),"""Jangan biarkan kekhawatiran menghalangi langk...","Gangguan Kecemasan: Jenis, Gejala, dan Cara Me...",Breathing for Inner Peace
3,Stress,Mindfulness,"""Jangan terlalu keras pada dirimu sendiri. Ter...",6 Cara Mengatasi Stres yang Mudah dan Efektif,Mindful Breathing Session
4,Stress,Meditasi Pernafasan (Breath Awareness),"""Jangan kita hanya terpaku pada pintu yang ter...",Manajemen Stres: Hidup Sehat dengan Stres,Breathe as a Path to Tranquility


In [4]:
# Preprocess text data
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_first['Quotes'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df_first['Output'], test_size=0.2, random_state=42)

In [5]:
# Train RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

# Evaluate the model
y_pred_rf = rf_classifier.predict(X_test)
print("Random Forest Classifier:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))

Random Forest Classifier:
Accuracy: 0.0
Classification Report:
              precision    recall  f1-score   support

     Anxiety       0.00      0.00      0.00       2.0
  Depression       0.00      0.00      0.00       1.0
      Stress       0.00      0.00      0.00       0.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_first['Quotes'])
X_lstm = tokenizer.texts_to_sequences(df_first['Quotes'])
X_lstm = pad_sequences(X_lstm)

y_lstm = pd.get_dummies(df_first['Output']).values

X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=42)

In [8]:
lstm_model = Sequential()
lstm_model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=X_lstm.shape[1]))
lstm_model.add(LSTM(100))
lstm_model.add(Dense(4, activation='softmax'))

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=10, batch_size=32, validation_data=(X_test_lstm, y_test_lstm))

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 1.3840 - accuracy: 0.2500 - val_loss: 1.3851 - val_accuracy: 0.3333
Epoch 2/10
1/1 [==============================] - 0s 69ms/step - loss: 1.3686 - accuracy: 0.9167 - val_loss: 1.3889 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 54ms/step - loss: 1.3530 - accuracy: 0.8333 - val_loss: 1.3936 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 54ms/step - loss: 1.3366 - accuracy: 0.8333 - val_loss: 1.3993 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 54ms/step - loss: 1.3187 - accuracy: 0.8333 - val_loss: 1.4063 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 72ms/step - loss: 1.2984 - accuracy: 0.7500 - val_loss: 1.4151 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 102ms/step - loss: 1.2753 - accuracy: 0.7500 - val_loss: 1.4268 - val_accuracy: 0.0000e+00
Epoc

In [9]:
# Evaluate the LSTM model
y_pred_lstm_prob = lstm_model.predict(X_test_lstm)
y_pred_lstm = np.argmax(y_pred_lstm_prob, axis=1)

y_test_labels = np.argmax(y_test_lstm, axis=1)

print("\nLSTM Model:")
print("Accuracy:", accuracy_score(y_test_labels, y_pred_lstm))
print("Classification Report:")
print(classification_report(y_test_labels, y_pred_lstm))

1/1 [==============================] - 0s 471ms/step

LSTM Model:
Accuracy: 0.0
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           2       0.00      0.00      0.00       1.0
           3       0.00      0.00      0.00       0.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [22]:
# Rekomendasikan berdasarkan output
def recommend_based_on_output(output, num_recommendations=3):
    # Implement logic untuk merekomendasikan meditasi, quotes, dan artikel berdasarkan output
    # Misalnya, Anda bisa menggunakan data Anda yang sudah disiapkan untuk mencari rekomendasi yang sesuai.
    recommended_data = df_first[df_first['Output'] == output].head(num_recommendations)

    recommended_meditations = recommended_data['Meditation'].tolist()
    recommended_quotes = recommended_data['Quotes'].tolist()
    recommended_articles = recommended_data['Artikel'].tolist()
    recommended_audios = recommended_data['Audio'].tolist()

    return recommended_meditations, recommended_quotes, recommended_articles, recommended_audios

In [23]:
# Rekomendasikan berdasarkan output RandomForest
recommended_rf = recommend_based_on_output('Stress', num_recommendations=3)

# Rekomendasikan berdasarkan output LSTM
#recommended_lstm = recommend_based_on_output('Stress', num_recommendations=3)

print("Rekomendasi RandomForest:")
for idx, (meditation, quote, article, audio) in enumerate(zip(*recommended_rf)):
    print(f"\nRekomendasi {idx + 1}:\nMeditasi: {meditation}\nQuotes: {quote}\nArtikel: {article}\nAudio: {audio}")

Rekomendasi RandomForest:

Rekomendasi 1:
Meditasi: Mindfulness
Quotes: "Jangan terlalu keras pada dirimu sendiri. Terkadang, memberi ruang untuk kesalahan adalah langkah pertama menuju pengampunan diri."
Artikel: 6 Cara Mengatasi Stres yang Mudah dan Efektif
Audio: Mindful Breathing Session

Rekomendasi 2:
Meditasi: Meditasi Pernafasan (Breath Awareness)
Quotes: "Jangan kita hanya terpaku pada pintu yang tertutup, sehingga tidak melihat masih ada pintu lain yang terbuka."
Artikel: Manajemen Stres: Hidup Sehat dengan Stres
Audio: Breathe as a Path to Tranquility

Rekomendasi 3:
Meditasi: Metta (kasih sayang)
Quotes: "Semuanya dimulai dengan pikiranmu. Ketika kamu mengubah pikiranmu, kamu mengubah kualitas hidupmu."
Artikel: Stress dan Penyebabnya
Audio: Heartfelt Embrace
